## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [72]:
#review number of cases
application_df.shape[0]


34299

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN","NAME"])

In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [24]:
#NEW Looking at ask amount which has the most unique variables. I want to convert this into ranges so that the data is more managable to predict with.
print(application_df["ASK_AMT"].min())
print(application_df["ASK_AMT"].max())
print(application_df["ASK_AMT"].median())
print(application_df["ASK_AMT"].mean())


5000
8597806340
5000.0
2769198.6820607013


In [33]:
#create a new list of ranges that captures all the values of ask amount and puts them into a range bracket

list_ranges = []

for n in range(len(application_df)):
    if application_df.loc[n, "ASK_AMT"] <= 5000:
        list_ranges.append("5K or less")
    elif application_df.loc[n, "ASK_AMT"] <= 10000:
        list_ranges.append("MT 5K to 10K")
    elif application_df.loc[n, "ASK_AMT"] <= 25000:
        list_ranges.append("MT 10K to 25K")
    elif application_df.loc[n, "ASK_AMT"] <= 50000:
        list_ranges.append("MT 25K to 50K")  
    elif application_df.loc[n, "ASK_AMT"] <= 100000:
        list_ranges.append("MT 50K to 100K")
    elif application_df.loc[n, "ASK_AMT"] <= 250000:
        list_ranges.append("MT 100K to 250K")
    elif application_df.loc[n, "ASK_AMT"] <= 500000:
        list_ranges.append("MT 250K to 500K")
    elif application_df.loc[n, "ASK_AMT"] <= 1000000:
        list_ranges.append("MT 500K to 1M")
    else:
        list_ranges.append("Over 1M")
        

#Append the list to a new column of the tables
application_df["ASK_AMT_RANGE"] = list_ranges

#check how many unique values in each range
application_df["ASK_AMT_RANGE"].value_counts()


ASK_AMT_RANGE
5K or less         25398
Over 1M             1577
MT 100K to 250K     1441
MT 50K to 100K      1423
MT 25K to 50K       1263
MT 10K to 25K       1135
MT 250K to 500K      863
MT 500K to 1M        650
MT 5K to 10K         549
Name: count, dtype: int64

In [34]:
#drop the Ask_amt column
application_df = application_df.drop(columns="ASK_AMT")
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL,ASK_AMT_RANGE
0,T10,Independent,C1000,ProductDev,Association,1,0,N,1,5K or less
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,1,MT 100K to 250K
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,0,5K or less
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,1,MT 5K to 10K
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,1,MT 100K to 250K


In [35]:
# Look at APPLICATION_TYPE value counts for binning
app_type = application_df["APPLICATION_TYPE"].value_counts()

app_type

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [36]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#I decided to use 500 as my cut off point. T9 was possible to include but seemed too low since the count for T3 is so high. Also the sample output has us start at 528 and leaves us with a nicely sized Other bun that is not too large. 
application_types_to_replace = list(app_type[app_type < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [37]:
# Look at CLASSIFICATION value counts for binning
classif = application_df["CLASSIFICATION"].value_counts()
classif

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [38]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classif.loc[classif >1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [39]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#Cut off at 1000 given the starter example. This also makes a reasonably sized other bucket. C7000 could reasonably be left out of the Other. 
classifications_to_replace = list(classif[classif < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [40]:
#checking data types of each column for the next step
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
IS_SUCCESSFUL              int64
ASK_AMT_RANGE             object
dtype: object

In [41]:
# Convert categorical data to numeric with `pd.get_dummies`

app_dummies_df = pd.get_dummies(application_df, dtype=int)
app_dummies_df.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_RANGE_5K or less,ASK_AMT_RANGE_MT 100K to 250K,ASK_AMT_RANGE_MT 10K to 25K,ASK_AMT_RANGE_MT 250K to 500K,ASK_AMT_RANGE_MT 25K to 50K,ASK_AMT_RANGE_MT 500K to 1M,ASK_AMT_RANGE_MT 50K to 100K,ASK_AMT_RANGE_MT 5K to 10K,ASK_AMT_RANGE_Over 1M
0,1,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [42]:
# Split our preprocessed data into our features and target arrays
#Chose is successful as the "y" value because all other values can be potential factors of success
y = app_dummies_df["IS_SUCCESSFUL"].values
X = app_dummies_df.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [43]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [45]:
#New model uses updated data set
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 80)                4160      
                                                                 
 dense_11 (Dense)            (None, 30)                2430      
                                                                 
 dense_12 (Dense)            (None, 1)                 31        
                                                                 
Total params: 6621 (25.86 KB)
Trainable params: 6621 (25.86 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [46]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [47]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5720 - accuracy: 0.7187
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5543 - accuracy: 0.7307
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5501 - accuracy: 0.7312
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5480 - accuracy: 0.7332
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5471 - accuracy: 0.7339
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5456 - accuracy: 0.7345
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5440 - accuracy: 0.7357
Epoch 8/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5428 - accuracy: 0.7364
Epoch 9/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5427 - accuracy: 0.7371
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5420 - accura

In [49]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5722 - accuracy: 0.7221 - 636ms/epoch - 2ms/step
Loss: 0.5722477436065674, Accuracy: 0.7220991253852844


In [90]:
#Reverting data back to how it was for the first model

application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

In [91]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN","NAME"])

In [92]:
# Look at APPLICATION_TYPE value counts for binning
app_type = application_df["APPLICATION_TYPE"].value_counts()

application_types_to_replace = list(app_type[app_type < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [93]:
# Look at CLASSIFICATION value counts for binning
classif = application_df["CLASSIFICATION"].value_counts()

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#Cut off at 1000 given the starter example. This also makes a reasonably sized other bucket. C7000 could reasonably be left out of the Other. 
classifications_to_replace = list(classif[classif < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [94]:
# Convert categorical data to numeric with `pd.get_dummies`

app_dummies_df = pd.get_dummies(application_df, dtype=int)
app_dummies_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [95]:
# Split our preprocessed data into our features and target arrays
#Chose is successful as the "y" value because all other values can be potential factors of success
y = app_dummies_df["IS_SUCCESSFUL"].values
X = app_dummies_df.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [96]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [97]:
#Change #2 Upping the number of nodes per layer

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  200
hidden_nodes_layer2 = 100


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_47 (Dense)            (None, 380)               16720     
                                                                 
 dense_48 (Dense)            (None, 200)               76200     
                                                                 
 dense_49 (Dense)            (None, 1)                 201       
                                                                 
Total params: 93121 (363.75 KB)
Trainable params: 93121 (363.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [98]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [99]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5681 - accuracy: 0.7236
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5555 - accuracy: 0.7301
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5527 - accuracy: 0.7323
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5521 - accuracy: 0.7318
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5507 - accuracy: 0.7332
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5491 - accuracy: 0.7331
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5482 - accuracy: 0.7346
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5481 - accuracy: 0.7346
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5475 - accuracy: 0.7358
Epoch 10/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5462 - accura

In [100]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5662 - accuracy: 0.7247 - 646ms/epoch - 2ms/step
Loss: 0.5661711692810059, Accuracy: 0.7246647477149963


In [107]:
#Adding a third hidden layer and adjusting the node amounts
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 8


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_58 (Dense)            (None, 100)               4400      
                                                                 
 dense_59 (Dense)            (None, 30)                3030      
                                                                 
 dense_60 (Dense)            (None, 8)                 248       
                                                                 
 dense_61 (Dense)            (None, 1)                 9         
                                                                 
Total params: 7687 (30.03 KB)
Trainable params: 7687 (30.03 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [108]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [109]:
# Train the model - Also upping # of epochs
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
804/804 [==============================] - 3s 2ms/step - loss: 0.5693 - accuracy: 0.7223
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5536 - accuracy: 0.7305
Epoch 3/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5509 - accuracy: 0.7325
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5499 - accuracy: 0.7319
Epoch 5/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5489 - accuracy: 0.7323
Epoch 6/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5480 - accuracy: 0.7349
Epoch 7/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5469 - accuracy: 0.7353
Epoch 8/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5468 - accuracy: 0.7349
Epoch 9/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5463 - accuracy: 0.7350
Epoch 10/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5453 - accura

In [110]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5628 - accuracy: 0.7240 - 493ms/epoch - 2ms/step
Loss: 0.5628145337104797, Accuracy: 0.7239649891853333


In [111]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimized.h5')

c:\Users\katyo\anaconda3\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
